In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import pickle

In [ ]:
X_full = pd.read_csv("../input/titanic/train.csv")
X_test = pd.read_csv("../input/titanic/test.csv")
ids = X_test['PassengerId']
ids.shape
X_test.shape

In [ ]:
X_full.isnull().sum()

In [ ]:
X_test.isnull().sum()

In [ ]:
#Substitua os valores nulos por um valor que não esteja presente naquela coluna dos dados, 
#se o seu modelo for de árvores de decisão. Dessa maneira você está informando o modelo que aquele valor é diferente de todos os outros.
#Filho, Mario. Manual Prático de Data Science (pp. 66-67). Kindle Edition. 

X_test.fillna(-9999, inplace=True)
X_full.fillna(-9999, inplace=True)

In [ ]:
# Train Data & Target
X = X_full.drop(['Survived'],axis=1)
y = X_full['Survived']

cat_cols = X_full.select_dtypes(include='object').columns.values
num_cols = X_full.select_dtypes(exclude='object').columns.values
X.head()

In [ ]:
#choose the features we want to train, just forget the float data
cate_features_index = np.where(X.dtypes != float)[0]
X.info()

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(X,y,random_state=42,train_size=.85)
X.columns.values[cate_features_index]

In [ ]:
model = CatBoostClassifier(eval_metric='Accuracy',use_best_model=True,random_seed=42)

In [ ]:
model.fit(X_train,y_train,cat_features=cate_features_index,eval_set=(X_val,y_val))

In [ ]:
test_preds = model.predict(X_test)

In [ ]:
submission = pd.DataFrame({
    'PassengerId':ids,
    'Survived': test_preds
})

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head(20)

In [26]:
# Salve o modelo
with open("modelo.pkl", "wb") as f:
    pickle.dump(model, f)

In [28]:
sample_input = X_full.loc[0]

In [31]:
sample_input = list(sample_input)

In [32]:
sample_input

[1,
 0,
 3,
 'Braund, Mr. Owen Harris',
 'male',
 22.0,
 1,
 0,
 'A/5 21171',
 7.25,
 -9999,
 'S']